# === Онлайн-рекомендации

Шаг 1. Набор похожих объектов
Чтобы получить набор похожих объектов, можно воспользоваться уже известным алгоритмом ALS из библиотеки implicit, у которого на такой случай есть удобный метод similar_items (подробнее о нём вы можете прочитать в официальной документации).
Воспользуемся им и получим по 10 самых похожих айтемов.
Задание 1 из 6
Дополните код ниже, чтобы получить набор похожих объектов в similar_items. Вы можете подглядеть решение в уроке «Коллаборативная фильтрация: ALS» — там вы реализовывали похожую логику для получения персональных рекомендаций.

In [1]:
import numpy as np
import pandas as pd

items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")
events_train = pd.read_parquet("events_train.par")
events_test = pd.read_parquet("events_test.par")

In [2]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])

events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

# создаём sparse-матрицу формата CSR 
events_train["rating"] = events_train["rating"].astype(int)
events_train["user_id_enc"] = events_train["user_id_enc"].astype(int)
events_train["item_id_enc"] = events_train["item_id_enc"].astype(int)
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

# создадим ALS-модель. Для примера возьмём количество латентных факторов для матриц $P, Q$, равным 50. 
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

als_model.similar_items(3)

/home/mle-user/anaconda3/lib/python3.12/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/home/conda/feedstock_root/build_artifacts/implicit_1724419594932/work/./implicit/gpu/utils.h:71)'
  warnings.warn(
/home/mle-user/anaconda3/lib/python3.12/site-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()


  0%|          | 0/50 [00:00<?, ?it/s]

(array([    3,  1942,     4,     1,     0,     2,  8250, 26800, 23781,
        18446], dtype=int32),
 array([0.9999999 , 0.9975803 , 0.9962052 , 0.98827547, 0.982088  ,
        0.97494876, 0.97159123, 0.90773606, 0.90773606, 0.82279867],
       dtype=float32))

In [3]:
# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

In [11]:
print(len(train_item_ids_enc))
print(len(similar_items[0]))

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

41474
41474


In [12]:
similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()})

similar_items.head()

,item_id_enc,sim_item_id_enc,score
0,611,"[611, 614, 613, 612, 616, 607, 619, 145, 610, ...","[1.000000238418579, 0.9779050350189209, 0.9777..."
1,783,"[783, 786, 784, 788, 785, 24929, 10687, 27653,...","[0.9999998807907104, 0.9860629439353943, 0.974..."
2,35259,"[35259, 34934, 39686, 28453, 32467, 32468, 385...","[1.0000001192092896, 0.9558345079421997, 0.822..."
3,40673,"[40673, 42197, 41790, 34293, 32096, 42363, 272...","[1.0000001192092896, 0.9654812216758728, 0.957..."
4,35502,"[35502, 23425, 33084, 31099, 1628, 39574, 1493...","[1.0000001192092896, 0.8471803069114685, 0.829..."


In [14]:
similar_items["sim_item_id_enc"].iloc[0]

[611, 614, 613, 612, 616, 607, 619, 145, 610, 609, 615]

In [16]:
similar_items = similar_items.explode(["sim_item_id_enc", "score"], ignore_index=True)
similar_items.head()

,item_id_enc,sim_item_id_enc,score
0,611,611,1.0
1,611,614,0.977905
2,611,613,0.977742
3,611,612,0.97427
4,611,616,0.974112


In [17]:
# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype("int")
similar_items["score"] = similar_items["score"].astype("float")

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"])
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"])
similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

In [18]:
similar_items.head()

,score,item_id_1,item_id_2
1,0.977905,5350,5354
2,0.977742,5350,5352
3,0.974270,5350,5351
4,0.974112,5350,5356
5,0.971654,5350,5346


In [19]:
similar_items[similar_items["item_id_1"]==7126]

,score,item_id_1,item_id_2
10495,0.948723,7126,7190
10496,0.940994,7126,24280
10497,0.930144,7126,1953
10498,0.925065,7126,58696
10499,0.916338,7126,38296
10500,0.916010,7126,2932
10501,0.913950,7126,7184
10502,0.911433,7126,387749
10503,0.909870,7126,7733
10504,0.909450,7126,30597


In [20]:
similar_items.to_parquet("similar_items.parquet") 

In [3]:
import pandas as pd
similar_items = pd.read_parquet("similar_items.parquet")
similar_items.set_index("item_id_1").loc[5350]



,score,item_id_2
item_id_1,,
5350,0.977905,5354
5350,0.977742,5352
5350,0.974270,5351
5350,0.974112,5356
5350,0.971654,5346
5350,0.970276,5359
5350,0.962662,1110
5350,0.959187,5349
5350,0.957078,5348


In [21]:
# Полезно убедиться, что полученный набор действительно содержит похожие данные. 
# Например, можно оценить глазами списки похожих объектов для каких-то уже известных.

def print_sim_items(item_id, similar_items):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si)

In [24]:
print_sim_items(7144, similar_items) 

,item_id,author,title,genre_and_votes,average_rating,ratings_count
1909078,7144,"Fyodor Dostoyevsky, David McDuff, Fyodor Dosto...",Crime and Punishment,"{'Classics': 15812, 'Fiction': 8028, 'Cultural...",4.19,390293


,score,item_id_1,item_id_2,author,title,genre_and_votes,average_rating,ratings_count
4632,0.964478,7144,12505,"Fyodor Dostoyevsky, Anna Brailovsky, Constance...",The Idiot,"{'Classics': 4036, 'Fiction': 2576}",4.18,76392
4633,0.953917,7144,12857,"Fyodor Dostoyevsky, Constance Garnett",The Gambler,"{'Classics': 946, 'Fiction': 729, 'Cultural-Ru...",3.88,22024
4634,0.952009,7144,67326,Fyodor Dostoyevsky,Poor Folk,"{'Classics': 320, 'Fiction': 235, 'Literature-...",3.73,4957
4635,0.946847,7144,5508624,Leo Tolstoy,Family Happiness,"{'Classics': 140, 'Fiction': 112, 'Cultural-Ru...",3.85,3337
4636,0.939763,7144,4934,"Fyodor Dostoyevsky, Fyodor Dostoyevsky, Richar...",The Brothers Karamazov,"{'Classics': 7496, 'Fiction': 5491, 'Cultural-...",4.31,158410
4637,0.938017,7144,17877,"Fyodor Dostoyevsky, Constance Garnett",The House of the Dead,"{'Classics': 533, 'Fiction': 441, 'Cultural-Ru...",4.04,8548
4638,0.937008,7144,929782,"Jack London, Andrew Sinclair",Martin Eden,"{'Classics': 435, 'Fiction': 405, 'Literature-...",4.39,13257
4639,0.936361,7144,28382,Nikolai Gogol,Diary of a Madman and Other Stories,"{'Classics': 284, 'Fiction': 243, 'Short Stori...",4.09,6241
4640,0.936321,7144,17690,"Franz Kafka, Max Brod, Willa Muir, Edwin Muir",The Trial,"{'Classics': 4607, 'Fiction': 4173, 'Literatur...",3.98,135862
4641,0.934541,7144,63038,Victor Hugo,The Man Who Laughs,"{'Classics': 352, 'Fiction': 176, 'Cultural-Fr...",4.22,5449


In [ ]:
# Шаг 2. Сервис Feature Store
# Теперь сделаем так, чтобы набор стал доступен сервису рекомендаций. 
# Для этого создадим новый сервис, который при запуске будет загружать набор похожих объектов из файла "similar_items.parquet" 
# и отдавать список похожих объектов через метод /similar_items.